# Clustering Neighborhoods in Toronto  
Welcome to this notebook. Here I'd like to cluster the neighborhoods in Toronto based on wikipedia's page https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M.  
The method I'll use will be K-Means Clustering. Before that, we need to preprocess the data and visualize it to make sure everything i do was right. So, let's go import the dataset

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Scraping Wikipedia page using BeautifulSoup

In [2]:
import requests
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M' # Wikipedia URL
html = requests.get(url).text # HTML Request
soup = BeautifulSoup(html) # Make an object of BeautifulSoup with source code from html variable

In [3]:
data = []
table = soup.find('table', attrs={'class':'wikitable'}) # Find class wikitable from the wikipedia page
table_body = table.find('tbody') # Find the body of the table

rows = table_body.find_all('tr') # Get all rows from the table
for row in rows:
    cols = row.find_all('td') # For each row, find all of the values on that row
    cols = [ele.text.strip() for ele in cols] # Eliminate empty spaces or tabs on values
    data.append([ele for ele in cols if ele]) # Get rid of empty values
df = pd.DataFrame(data, columns=['PostalCode','Borough','Neighborhood']) # Create new dataframe
df.drop([0],inplace=True) # Drop first row
df.head()

,PostalCode,Borough,Neighborhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront


## Preprocessing

So, thanks to BeautifulSoup now I can start preprocessing the dataset. Here are the things I need to do for the preprocesing step:  
1. Ignore cells with a Borough that is NOT ASSIGNED  
2. If there are multiple Neighborhood on the same PostalCode, these two Neighborhood will be combined into the same row  
3. If a row has a borough but a NOT ASSIGNED Neighborhood, then the Neighborhood will be the same as the Borough  

Okay that's all we need to do. Let's get started

### Ignore cells with borough equals NOT ASSIGNED

In [4]:
df = df[df['Borough']!='Not assigned']
df.head(10)

,PostalCode,Borough,Neighborhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Downtown Toronto,Queen's Park
10,M9A,Queen's Park,Not assigned
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern
14,M3B,North York,Don Mills North


### Combine multiple neighborhoods with the same PostalCode

In [5]:
df = df.groupby(['PostalCode','Borough']).agg({'Neighborhood' : lambda x: ', '.join(x)}).reset_index()
df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### When the row have Neighborhood as NOT ASSIGNED, change it into the name of the Borough

In [6]:
df['Neighborhood'] = np.where(np.equal(df['Neighborhood'],'Not assigned'), df['Borough'],df['Neighborhood'])
df[df['Borough']==df['Neighborhood']]

,PostalCode,Borough,Neighborhood
93,M9A,Queen's Park,Queen's Park


In [7]:
df.shape

(103, 3)

Okay now the data is ready. The next step is retrieving location data. Now I'm going to do this using Geocoder. Let's get to it

## Retrieve Location Data

In [8]:
!pip install geocoder

In [9]:
#from geopy.geocoders import Nominatim
#df['lat'] = None
#df['lon'] = None
#for index, row in df.iterrows():
#    address = '{}, Toronto, Ontario'.format(row['PostalCode'])
#    geolocator = Nominatim(user_agent="toro_explorer")
#    location = geolocator.geocode(address)
#    print(row['PostalCode'])
#    print(location)
#    df.iloc[index,3] = location.latitude
#    df.iloc[index,4] = location.longitude

So i'm kinda stuck in here. Then I'll just get the dataset given by the instructor

In [10]:
#df.drop(['lat','lon'],axis=1,inplace=True)

In [11]:
!wget -q -O 'location_toronto.csv' https://cocl.us/Geospatial_data

In [12]:
df_location = pd.read_csv('location_toronto.csv')
df_location.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [13]:
merged_df = pd.merge(df,df_location, left_on='PostalCode', right_on='Postal Code')
merged_df.drop("Postal Code",axis=1,inplace=True)
merged_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


# Visualize Map using Folium  
The next step is visualizing the map using Folium. Let's do it

In [14]:
!pip install folium

In [15]:
import folium
latitude = 43.651070
longitude = -79.347015

In [16]:
# create map of New York using latitude and longitude values
toronto_map = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(merged_df['Latitude'], merged_df['Longitude'], merged_df['Borough'], merged_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(toronto_map)  
    
toronto_map

Alright visualizing map successful! And I'm seeing that most of the appearing points are near to the middle-bottom area. That must be the center of Toronto or most people would live there. But no worries about that, let's now cluster these neighborhoods!

## Clustering by Distance  
I'm going to cluster the neighborhood by distance only now

In [17]:
from sklearn.cluster import KMeans
k = 4 # number of clusters  
grouped_clustering = merged_df.drop(['PostalCode','Borough','Neighborhood'],axis=1)
kmeans = KMeans(n_clusters=k, random_state=0).fit(grouped_clustering)

In [18]:
merged_df.insert(0, 'Cluster Labels', kmeans.labels_)
#grouped = merged_df.join(merged_df.set_index('Neighborhood'), on='Neighborhood')
merged_df.head()

,Cluster Labels,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,2,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,2,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,2,M1G,Scarborough,Woburn,43.770992,-79.216917
4,2,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [19]:
import matplotlib.cm as cm
import matplotlib.colors as colors
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(merged_df['Latitude'], merged_df['Longitude'], merged_df['Neighborhood'], merged_df['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

So that's it. If I'm going to cluster with 4 groups, then the above map describes it all. Remember, I only cluster it by distance only.  
Finally, I'm going to explore each neighborhood using Foursquare API, and the cluster it again based on the exploration

## Cluster with Features

In [20]:
# @hidden_cell
CLIENT_ID = 'RHI5IEPT0XWCSGZMOPGTUGBAEM2JS5HYTCJAVN5XOYJVVO2P' # your Foursquare ID
CLIENT_SECRET = 'HVKSYMEKAT5NL03QHX15Q0M4NZL4K3MKM3M3ZRNYIZDR0ZTW' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [21]:
merged_df.head()

,Cluster Labels,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,2,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,2,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,2,M1G,Scarborough,Woburn,43.770992,-79.216917
4,2,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [22]:
new_merged_df = merged_df.drop('Cluster Labels',axis=1)
neighborhood_latitude = new_merged_df.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = new_merged_df.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = new_merged_df.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Rouge, Malvern are 43.806686299999996, -79.19435340000001.


In [23]:
LIMIT = 100
radius = 500

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=RHI5IEPT0XWCSGZMOPGTUGBAEM2JS5HYTCJAVN5XOYJVVO2P&client_secret=HVKSYMEKAT5NL03QHX15Q0M4NZL4K3MKM3M3ZRNYIZDR0ZTW&v=20180605&ll=43.806686299999996,-79.19435340000001&radius=500&limit=100'

In [24]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e2d2629760a7f001b467f33'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Malvern',
  'headerFullLocation': 'Malvern, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 1,
  'suggestedBounds': {'ne': {'lat': 43.8111863045, 'lng': -79.18812958073042},
   'sw': {'lat': 43.80218629549999, 'lng': -79.2005772192696}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bb6b9446edc76b0d771311c',
       'name': "Wendy's",
       'location': {'crossStreet': 'Morningside & Sheppard',
        'lat': 43.80744841934756,
        'lng': -79.19905558052072,
        'labeledLatLngs': [{'label': 'display'

In [25]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [26]:
from pandas.io.json import json_normalize
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Wendy's,Fast Food Restaurant,43.807448,-79.199056


In [27]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

1 venues were returned by Foursquare.


The above neighborhood only returns 1 venue, which is Wendy's, a Fast Food Restaurant

In [28]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        try:
            results = requests.get(url).json()["response"]['groups'][0]['items']
            venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
        except:
            print("An exception occurred")    

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [29]:
venues = getNearbyVenues(names=new_merged_df['Neighborhood'],
                                   latitudes=new_merged_df['Latitude'],
                                   longitudes=new_merged_df['Longitude']
                                  )

In [30]:
print(venues.shape)
venues.head()

(2215, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant


In [31]:
venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Agincourt,4,4,4,4,4,4
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",2,2,2,2,2,2
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",11,11,11,11,11,11
"Alderwood, Long Branch",9,9,9,9,9,9
"Bathurst Manor, Downsview North, Wilson Heights",20,20,20,20,20,20
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",23,23,23,23,23,23
Berczy Park,56,56,56,56,56,56


In [32]:
# one hot encoding
onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
onehot['Neighborhood'] = venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [onehot.columns[-1]] + list(onehot.columns[:-1])
onehot = onehot[fixed_columns]
print(onehot.shape)
onehot.head()

(2215, 268)


,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [33]:
grouped = onehot.groupby('Neighborhood').mean().reset_index()
print(grouped.shape)
grouped.head()

(100, 268)


,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,"Adelaide, King, Richmond",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.02,...,0.0,0.02,0.0,0.000000,0.0,0.0,0.01,0.0,0.0,0.01
1,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.00,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.00
2,"Agincourt North, L'Amoreaux East, Milliken, St...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.00,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.00
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.00,0.0,0.090909,0.0,0.0,0.00,0.0,0.0,0.00
4,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.00,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.00


In [34]:
num_top_venues = 5

for hood in grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = grouped[grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
         venue  freq
0  Coffee Shop  0.07
1         Café  0.04
2   Steakhouse  0.04
3          Bar  0.04
4   Restaurant  0.03


----Agincourt----
                       venue  freq
0  Latin American Restaurant  0.25
1             Breakfast Spot  0.25
2               Skating Rink  0.25
3                     Lounge  0.25
4                Yoga Studio  0.00


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
                             venue  freq
0                       Playground   0.5
1                             Park   0.5
2                      Yoga Studio   0.0
3               Mexican Restaurant   0.0
4  Molecular Gastronomy Restaurant   0.0


----Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown----
                  venue  freq
0         Grocery Store  0.18
1           Pizza Place  0.09
2          Liquor Store  0.09
3              Pharmacy  0.09
4  Fast Food Restaurant  0.0

In [35]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [36]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = grouped['Neighborhood']

for ind in np.arange(grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Steakhouse,Bar,Asian Restaurant,Restaurant,Breakfast Spot,Hotel,Thai Restaurant,Gastropub
1,Agincourt,Latin American Restaurant,Lounge,Skating Rink,Breakfast Spot,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Park,Playground,Women's Store,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Drugstore
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Coffee Shop,Pizza Place,Fried Chicken Joint,Sandwich Place,Beer Store,Fast Food Restaurant,Video Store,Pharmacy,Liquor Store
4,"Alderwood, Long Branch",Pizza Place,Gym,Coffee Shop,Pharmacy,Skating Rink,Athletics & Sports,Pub,Sandwich Place,Discount Store,Dessert Shop


Now the clustering of those features

In [52]:
from sklearn.cluster import KMeans
k = 4 # number of clusters  
new_grouped = grouped.drop('Neighborhood',axis=1)
kmeans = KMeans(n_clusters=k, random_state=0).fit(new_grouped)

In [53]:
new_grouped.insert(0, 'Cluster Labels', kmeans.labels_)
new_grouped = pd.concat([merged_df.drop('Cluster Labels',axis=1), new_grouped], axis=1, sort=False)
new_grouped.dropna(inplace=True)
new_grouped['Cluster Labels'] = new_grouped['Cluster Labels'].astype('int32')
new_grouped.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,1,0.0,0.0,0.0,0.0,...,0.0,0.02,0.0,0.000000,0.0,0.0,0.01,0.0,0.0,0.01
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,1,0.0,0.0,0.0,0.0,...,0.0,0.00,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.00
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.00
3,M1G,Scarborough,Woburn,43.770992,-79.216917,1,0.0,0.0,0.0,0.0,...,0.0,0.00,0.0,0.090909,0.0,0.0,0.00,0.0,0.0,0.00
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,1,0.0,0.0,0.0,0.0,...,0.0,0.00,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.00


In [54]:
import matplotlib.cm as cm
import matplotlib.colors as colors
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(new_grouped['Latitude'], new_grouped['Longitude'], new_grouped['Neighborhood'], new_grouped['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Finally it's done. Feel free to ask or give me a feedback. Thank you for checking out my notebook!